In [1]:
import numpy as np
import scipy as sc
import open3d as o3d
from math import sqrt
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import euclidean_distances

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
PLY = "./models/1.ply"

print("Abriendo point cloud")
pcd = o3d.io.read_point_cloud(PLY)
print("Abriendo malla")
mesh = o3d.io.read_triangle_mesh(PLY)

Abriendo point cloud
Abriendo malla


# Perímetro

Se calcula el perimetro como la suma de las distancia Euclideana de los puntos de la frontera de la ulcera. Para ello se calculan los puntos que pertenecen a la envoltura convexa de la nube de puntos y se calcula la distancia entre ellos. 

- Fuente: Wound 3D Geometrical Feature Estimation Using Poisson Reconstruction

In [3]:
def perimeter(ulcer_pts):
    points_boundary = np.asarray(ulcer_pts.select_by_index(ulcer_pts.compute_convex_hull()[1]).points)
    distances = euclidean_distances(points_boundary, points_boundary)
    p = distances[0][-1]
    for i in range(0, distances.shape[0]-1):
        p += distances[i][i+1]
    return p


In [4]:
print("Perímetro:", perimeter(pcd), "u")

Perímetro: 1.1269077642421501 u


# Área

El área se calcula como la suma de todos los triángulos de la malla utilizando la fórmula de Herón.

$A_t = \sqrt{s(s-a)(s-b)(s-c)}$

donde $s = \frac{P_t}{2}$ y $P_t$ es el perímetro del triángulo.

Entonces $A_M = \sum_{i = 0}^n A_{t_i}$ donde $M$ es la malla con $n$ triángulos.

In [5]:
def heron(p):
    S = euclidean_distances(p, p)
    SP = (S[0][1] + S[0][2] + S[1][2])/2
    return sqrt(SP * (SP - S[0][1]) * (SP - S[0][2]) * (SP - S[1][2]))

def area(ulcer_pts, ulcer_mesh):
    triangles = np.asarray(ulcer_mesh.triangles)
    points = np.asarray(ulcer_pts.points)
    area = 0
    for t in triangles:
        pts = points[t]
        area += heron(pts)
    return area

In [6]:
print("Area:",area(pcd, mesh), "u2")

Area: 0.0016225300427622836 u2


# Volumen

Para calcular el volumen se procede a generar la tapa de la ulcera utilizando spline cubico, luego se genera una triangulacion de Delaunay entre la tapa y la ulcera para luego calcular el volumen de cada piramide resultante de la triangulacion utilizando el area segun Heron y la distancia de punto a plano.

In [7]:
def get_plane(x: np.array, y: np.array, z: np.array):
    xy = y - x
    xz = z - x
    n = np.cross(xy, xz)
    d = np.dot(x, n)
    return np.concatenate((n, [-d]))

def pP_distance(p0: np.array, x: np.array, y: np.array, z: np.array):
    plane = get_plane(x, y, z)
    num = abs(np.dot(np.concatenate((p0, [1])), plane))
    den = sqrt(np.sum(plane[:3] ** 2))
    return num / den

def volume(ulcer_pts, save_top=True, name=""):
    # obtengo los puntos del borde de la ulcera
    points_boundary = np.asarray(pcd.select_by_index(ulcer_pts.compute_convex_hull()[1]).points)
    # inicializo un interpolador
    interpolate = sc.interpolate.CloughTocher2DInterpolator(points_boundary[:,:2], points_boundary[:,-1])
    #interpolate = sc.interpolate.NearestNDInterpolator(points_boundary[:,:2], points_boundary[:,-1])
    # calculo los maximos y los minimos de los puntos del borde para generar la tapa
    mins = np.min(points_boundary[:,:2], axis=0)
    maxs = np.max(points_boundary[:,:2], axis=0)
    x = np.linspace(mins[0], maxs[0], 50)
    y = np.linspace(mins[1], maxs[1], 50)
    x, y = np.meshgrid(x, y)
    points = np.dstack((x, y))
    # genero los puntos de la tapa usando la interpolacion
    points_3d = []
    for i in range(points.shape[0]):
        for j in range(points.shape[1]):
            points_3d.append([points[i][j][0], points[i][j][1], interpolate(points[i][j])[0]])
    points_3d = np.array(points_3d)
    points_3d = points_3d[~np.isnan(points_3d[:,-1])]
    if save_top:
        top = o3d.geometry.PointCloud()
        top.points = o3d.utility.Vector3dVector(points_3d)
        o3d.io.write_point_cloud(f"./models/tops/top_{name}.ply", top)
    # luego hago la triangulacion de Delaunay en 3D
    delaunay = sc.spatial.Delaunay(np.concatenate((points_3d, points_boundary)))
    # se calcula el volumen de cada piramide
    volume = 0
    for pyramid in delaunay.simplices:
        pts = delaunay.points[pyramid]
        AB = heron(pts[1:])
        h = pP_distance(pts[0], pts[1], pts[2], pts[3])
        volume += (AB * h) / 3
    return volume

In [288]:
print("Volumen",volume(pcd, name="1"), "u3")

Volumen 0.00012369791377811587 u3
